In [93]:
# edit for entire script to be included w/in function definition
def SPASE_Scraper(directory, relFilePath):
    import xml.etree.ElementTree as ET
    import os
    import pprint

    # establish path of XML file
    path = "../NASA/NumericalData/ACE/Attitude/Definitive/"
    fullname = os.path.join(path, "PT1H.xml")
    file_size = os.path.getsize(fullname)
    print("File size is: " + str(file_size) + " bytes")
    tree = ET.parse(fullname)
    root = tree.getroot()
    # root[1] = NumericalData
    # root = Spase
    # ET.dump(tree)
    # ET.register_namespace("spase","http://www.spase-group.org/data/schema")
    # ns = {"spase":"http://www.spase-group.org/data/schema"}

File size is: 13268 bytes


In [97]:
# iterate thru children to obtain ResourceID and locate ResourceHeader
for child in root[1]:
    if child.tag.endswith("ResourceID"):
        RID = child.text
    elif child.tag.endswith("ResourceHeader"):
        targetChild = child
    
# obtain Author, Publication Date, Publisher, Persistent Identifier, and Dataset Name

# define vars
author="None" 
pubDate="None"
pub = "None"
dataset = "None"
PI = "None"
# holds role values that are not considered for author var
UnapprovedAuthors = ["MetadataContact", "ArchiveSpecialist", "HostContact", "Publisher", "User"]

# iterate thru ResourceHeader
for child in targetChild:
    # find backup Dataset Name
    if child.tag.endswith("ResourceName"):
        targetChild = child
        dataset = child.text
    # find Persistent Identifier
    elif child.tag.endswith("DOI"):
        PI = child.text
    # find Publication Info
    elif child.tag.endswith("PublicationInfo"):
        PI_Child = child
        #break
    # find Contact
    elif child.tag.endswith("Contact"):
        C_Child = child
        # iterate thru Contact to find PersonID and Role
        for child in C_Child:
            # find PersonID
            if child.tag.endswith("PersonID"):
                # store PID
                PID = child.text
            # find Role
            elif child.tag.endswith("Role"):
                # backup author
                if child.text == "PrincipalInvestigator" or "PI":
                    author = PID
                # backup publisher
                elif child.text == "Publisher":
                    pub = child.text
                # backup author
                elif child.text not in UnapprovedAuthors:
                    author = PID

# access Publication Info
for child in PI_Child:
    if child.tag.endswith("Authors"):
        author = child.text
    elif child.tag.endswith("PublicationDate"):
        pubDate = child.text
    elif child.tag.endswith("PublishedBy"):
        pub = child.text
    elif child.tag.endswith("Title"):
        dataset = child.text

pubYear = pubDate[0:4]
print("The ResourceID is " + RID)
print("The author is " + author)
print("The publication year is " + pubYear)
print("The publisher is " + pub)
print("The dataset is " + dataset)
print("The persistent identifier is " + PI)

The ResourceID is spase://NASA/NumericalData/ACE/Attitude/Definitive/PT1H
The author is ACE flight operations team (FOT), and R. Candey
The publication year is 2022
The publisher is NASA Space Physics Data Facility
The dataset is ACE Definitive Attitude, Hourly Values, Direction Cosines in RTN, GSE, and J2000 GCI Coordinates
The persistent identifier is https://doi.org/10.48322/f2a8-nm36


In [95]:
# obtain data links and license

# dictionary labled by the Access Rights which will store all URLs and their Product Keys if given
Open = {}

# iterate thru children to locate Access Information
for child in root[1]:
    if child.tag.endswith("AccessInformation"):
        targetChild = child
        # iterate thru children to locate AccessURL and RepositoryID
        for child in targetChild:
            if child.tag.endswith("AccessURL"):
                targetChild = child
                # iterate thru children to locate URL
                for child in targetChild:
                    if child.tag.endswith("URL"):
                        url = child.text
                        Open[url] = "No_product_key"
                    # check if URL has a product key
                    elif child.tag.endswith("ProductKey"):
                        prodKey = child.text
                        Open[url] = prodKey
            # find backup Publisher if needed
            elif pub == "None":
                if child.tag.endswith("RepositoryID"):
                    # use partition to split text by Repository/ and assign only the text after it to pub 
                    before, sep, after = child.text.partition("Repository/")
                    pub = after
            # continue to check for additional AccessURLs            
            continue
    # continue to check for additional Access Informations
    continue

print("The data links and their associated product keys with an Open License are:")
pprint.pprint(Open)

The data links and their associated product keys with an Open License are:
{'ftps://spdf.gsfc.nasa.gov/pub/data/ace/orbit/level_2_cdaweb/def_at/': 'No_product_key',
 'https://cdaweb.gsfc.nasa.gov/cgi-bin/eval2.cgi?dataset=AC_AT_DEF&index=sp_phys': 'AC_AT_DEF',
 'https://cdaweb.gsfc.nasa.gov/hapi': 'AC_AT_DEF',
 'https://spdf.gsfc.nasa.gov/pub/data/ace/orbit/level_2_cdaweb/def_at/': 'No_product_key'}
